Условие: Задача состоит в модификации кода градиентного спуска для задачи регрессии (ноутбук лекции - Gradient_descent.ipynb). Необходимо, чтобы ваш градиентный спуск мог решать задачи классификации, а именно линейной классификации и логистической регрессии.
На всякий случай напомню, что код будет разный, поэтому жду от вас две функции градиентного спуска.
Чтобы решить задачи, обратите внимание на пункт “Принцип максимального правдоподобия” статьи http://www.machinelearning.ru/wiki/images/6/68/voron-ML-Lin.pdf . Там находится ответ на вопрос “какая функция потерь”.

In [1]:
# first generate a data
import random
import pandas as pd
import numpy as np

# Don't run this cell

In [2]:
data = random.sample(range(1, 1000), 100)
data = pd.DataFrame(data)

# simple class determ function. odd/even
data['guess'] = data[0].apply(lambda x: x % 2)

# Don't run this cell

In [3]:
# the model crashed while trying to predict this
# was predictable, because it is not linear function
# let's do it another way

data = random.sample(range(1, 1000), 100)
data = pd.DataFrame(data)

def if_more(x):
    if x > 500:
        return 1
    else:
        return 0

# simple class determ function. odd/even
data['guess'] = data[0].apply(if_more)
data

,0,guess
0,602,1
1,566,1
2,725,1
3,380,0
4,607,1
...,...,...
95,636,1
96,23,0
97,459,0
98,367,0


In [4]:
# run only if you ran the one of previous two cells
x = data
x = x.to_numpy()
y = data['guess']
x[:5]

array([[602,   1],
       [566,   1],
       [725,   1],
       [380,   0],
       [607,   1]], dtype=int64)

# Linear classifier

In [5]:
def gradient_descent_class(alpha, x, y, numIterations):
    """
    Linear gradient classifier
    """
    
    m = x.shape[0] 
    
    # add ones for delta
    x = np.c_[ np.ones(m), x]
    
    n = x.shape[1]
    
    theta = np.ones(n) # model weights
    x_transpose = x.transpose()
    
    print(f"Initializing gradient search. M is {m}. N is {n}")
    
    for iter in range( 0, numIterations ):
        hypothesis = np.dot(x, theta) # матричное произведение
        
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)  # функция потерь
        
        #print( "iter %s | J: %.3f" % (iter, J) )
        #print(theta)
        
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient  # update
    
    print('Model weights:')
    print(theta)
    return theta

In [6]:
# fuck it, take ready dataset
from sklearn import datasets
iris = datasets.load_iris()

prepare_data = pd.DataFrame(iris.data)
prepare_data['class'] = iris.target

# remove third class
prepare_data = prepare_data[prepare_data['class'] != 2] 

x = prepare_data.drop('class', axis=1)
x = x.to_numpy()
y = prepare_data['class']
y = y.to_numpy()

In [7]:
alpha = 0.01
theta = gradient_descent_class(alpha, x, y, 1000)

Initializing gradient search. M is 100. N is 5
Model weights:
[ 0.74630389 -0.25887133  0.04055461  0.17711531  0.68463828]


In [8]:
def class_prediction(row):
    global theta
    return theta[0] + theta[1]*row[0] + theta[2]*row[1] + theta[3]*row[2] + theta[4]*row[3]

results = pd.DataFrame(x)
results['class'] = y
results['predict'] = results.apply(class_prediction, axis=1)
results

,0,1,2,3,class,predict
0,5.1,3.5,1.4,0.2,0,-0.047110
1,4.9,3.0,1.4,0.2,0,-0.015613
2,4.7,3.2,1.3,0.2,0,0.026561
3,4.6,3.1,1.5,0.2,0,0.083816
4,5.0,3.6,1.4,0.2,0,-0.017167
...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851
96,5.7,2.9,4.2,1.3,1,1.022260
97,6.2,2.9,4.3,1.3,1,0.910536
98,5.1,2.5,3.0,1.1,1,0.811895


In [9]:
from sklearn import linear_model
model = linear_model.SGDRegressor( alpha = 0.01, max_iter = 1000 )

model.fit( x, y )

SGDRegressor(alpha=0.01, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [10]:
results['control'] = results.apply(lambda x: model.intercept_ + model.coef_[0]*x[0] 
                                   + model.coef_[1]*x[1] + model.coef_[2]*x[2] + model.coef_[3]*x[3], axis=1)

In [11]:
results

,0,1,2,3,class,predict,control
0,5.1,3.5,1.4,0.2,0,-0.047110,[-0.009035333990028177]
1,4.9,3.0,1.4,0.2,0,-0.015613,[0.05208301428620529]
2,4.7,3.2,1.3,0.2,0,0.026561,[-0.0016840380707527636]
3,4.6,3.1,1.5,0.2,0,0.083816,[0.06249858485815071]
4,5.0,3.6,1.4,0.2,0,-0.017167,[-0.022767497507691276]
...,...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.957851,[0.9055211394723255]
96,5.7,2.9,4.2,1.3,1,1.022260,[0.9290099924992103]
97,6.2,2.9,4.3,1.3,1,0.910536,[0.9607001959009007]
98,5.1,2.5,3.0,1.1,1,0.811895,[0.6358626162996959]


# Logistic regression
http://neerc.ifmo.ru/wiki/index.php?title=%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F

In [49]:
import math


def activation_f(x,w):
    """
    Returns target function for logistic classifier
    x - parameters
    w - weights of model
    1 / (1 + e ** -x*w.transpose) Скалярное произведение
    """
    e_power = -1 * np.dot(x, w.transpose())
    return 1 / (1 + np.exp(e_power)) #


def gdc_log(alpha, x, y, numIterations):
    """
    Logistic gradient classifier
    """
    
    m = x.shape[0] 
    
    # add ones for delta
    x = np.c_[ np.ones(m), x]
    
    # model weights
    n = x.shape[1]
    w = np.ones(n)
    
    print(f"Initializing gradient search. M is {m}. N is {n}. X:")
    print(x)
    
    for iter in range( 0, numIterations ):
        # гипотеза
        a = activation_f(x,w) 
        
        # loss function
        loss = (-1) * y * np.log(a) - (1-y) * np.log(1 - a)
        loss = loss.sum()
        print(f"ITERATION {iter} | LOSS: {loss} | ACTIVATION: {a}  W:{w}")
        gradient = np.dot((a - y), x) 
        w = w - alpha * gradient  # update
    
    print('Model weights:')
    print(w)
    return w

In [ ]:
alpha = 0.01
w = gdc_log(alpha, x, y, 1000)

In [14]:
# check results by standart libraries 
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs')
model = clf.fit(x, y)

In [15]:
def act(x,x_count,w):
    """
    Converts X and weights of model into prediction
    x_count - how much columns is X
    """
    
    x = x[:x_count]
    # add ones for delta
    x = np.concatenate([[1], x])
    
    return activation_f(x, w)

In [16]:
logr = pd.DataFrame(x)
logr['class'] = y

l = x.shape[1]
logr['predict'] = logr.apply(lambda x: act(x, l, w), axis=1)
logr['control'] = logr.apply(lambda x: act(x, l, np.concatenate([model.intercept_, model.coef_[0]])), axis=1)
logr

,0,1,2,3,class,predict,control
0,5.1,3.5,1.4,0.2,0,0.000042,0.016051
1,4.9,3.0,1.4,0.2,0,0.000294,0.022968
2,4.7,3.2,1.3,0.2,0,0.000113,0.014054
3,4.6,3.1,1.5,0.2,0,0.000553,0.023149
4,5.0,3.6,1.4,0.2,0,0.000035,0.014056
...,...,...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1,0.999922,0.982502
96,5.7,2.9,4.2,1.3,1,0.999959,0.985442
97,6.2,2.9,4.3,1.3,1,0.999954,0.990678
98,5.1,2.5,3.0,1.1,1,0.993001,0.794266


# Model scoring
## without using libs

In [31]:
def accurancy(y, p):
    tp = 0
    tn = 0
    fn = 0
    fp = 0
    
    for i in range(len(y)):
        if p[i]>0.5 and y[i] == 1:
            tp += 1
        elif p[i] > 0.5 and y[1] == 0:
            fp += 1
        elif p[i] < 0.5 and y[i] == 0:
            tn += 1
        else:
            fn += 1
    
    print(f'True positive: {tp} | True negative: {tn} | False positive: {fp} | False negative: {fn}')
    return (tp + tn) / (tp + tn + fp + fn)

acc = accurancy(y, logr['predict'])
print(f'My model accurancy {acc}')

acc = accurancy(y, logr['control'])
print(f'Standart score {acc}')            

True positive: 0 | True negative: 38896 | False positive: 0 | False negative: 34715
My model accurancy 0.5283992881498689
True positive: 25877 | True negative: 24426 | False positive: 14470 | False negative: 8838
Standart score 0.68336254092459


In [18]:
from sklearn.metrics import accuracy_score

def normalize(a):
    for i in range(len(a)):
        if a[i] > 0.5:
            a[i] = 1
        else:
            a[i] = 0
    
    return a
            
acc = accuracy_score(y, normalize(logr['predict']))
print(f'My model accurancy {acc}')

acc = accuracy_score(y, normalize(logr['control']))
print(f'Standart score {acc}')

My model accurancy 1.0
Standart score 1.0


D:\Python\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Python\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Perfect models... Need more data

In [19]:
taxi = pd.read_csv('taxi_log.csv')
taxi = taxi[taxi['distance_km'] >= 0]

In [20]:
X = taxi.drop('driver_response', axis=1).to_numpy()
y = taxi['driver_response'].to_numpy()
y

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [36]:
alpha = 0.01
w = gdc_log(alpha, X, y, 10)

clf = LogisticRegression(random_state=0, solver='lbfgs')
model = clf.fit(X, y)

Initializing gradient search. M is 73611. N is 6. X:
[[ 1.         55.805342   55.819329   18.802      25.217       3.42325973]
 [ 1.         55.813978   55.814827    6.747       9.8         0.41135558]
 [ 1.         55.803578   55.812559   12.383      19.25        1.06411596]
 ...
 [ 1.         55.961315   55.962203   46.983      46.217       0.20503258]
 [ 1.         55.961354   55.964892   50.029      51.1         0.39443479]
 [ 1.         55.961364   55.962322   54.811      56.183       0.43466928]]
ITERATION 0 | LOSS: nan | ACTIVATION: [1. 1. 1. ... 1. 1. 1.]  W:[1. 1. 1. 1. 1. 1.]
ITERATION 1 | LOSS: nan | ACTIVATION: [0. 0. 0. ... 0. 0. 0.]  W:[  -387.96       -21706.43002917 -21706.13715115 -12349.17551
 -14374.90921      -350.88923843]
ITERATION 2 | LOSS: nan | ACTIVATION: [0. 0. 0. ... 0. 0. 0.]  W:[  -40.81       -2353.04456785 -2352.94213245 -6055.1193
 -5366.62288       25.38706975]
ITERATION 3 | LOSS: nan | ACTIVATION: [1. 1. 1. ... 1. 1. 1.]  W:[  306.34       17000.3408

D:\Python\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
D:\Python\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in multiply


In [34]:
logr = pd.DataFrame(X)
l = X.shape[1]
w_model = np.concatenate([model.intercept_, model.coef_[0]])
logr['predict'] = logr.apply(lambda x: act(x, l, w), axis=1)

logr['control'] = logr.apply(lambda x: act(x, l, w_model), axis=1)
logr['class'] = y

acc = accurancy(y, logr['predict'].to_numpy())
print(f'My model accurancy {acc}.')
print(f'Weights {w}\n\n')


acc = accurancy(y, logr['control'])
print(f'Standart score {acc}')   
print(f'Weights {w_model}')

True positive: 0 | True negative: 38896 | False positive: 0 | False negative: 34715
My model accurancy 0.5283992881498689.
Weights [   -35.07        -2125.65167495  -2125.58013225 -18823.2915
 -15518.81394       266.98919021]


True positive: 25877 | True negative: 24426 | False positive: 14470 | False negative: 8838
Standart score 0.68336254092459
Weights [ 0.00147743  0.00303696  0.0025849  -0.05780542  0.02663625  0.12091457]


In [61]:
print(np.concatenate([[1],X[1]]))
print(w)
e_power = -1 * np.dot(np.concatenate([[1],X[1]]), w.transpose())
print(e_power)
1 / (1 + np.exp(e_power))

[ 1.         55.813978   55.814827    6.747       9.8         0.41135558]
[   -35.07        -2125.65167495  -2125.58013225 -18823.2915
 -15518.81394       266.98919021]
516290.3300459111


0.0

My function fails on this kind of dataset due to really huge power of epsilon in activation function